In [ ]:
### lade videos von S3 Speicher herunter und extrahiere Bilder aller 60 Frames ###

!pip install requirements.txt

from PIL import Image 
import cv2
import time
import os
from os.path import exists
import boto3


def get_s3_client():
    config = configparser.ConfigParser()
    config.read('config.txt')
    return boto3.client('s3',
                    aws_access_key_id=config['default']['aws_access_key_id'],
                    aws_secret_access_key=config['default']['aws_secret_access_key'],
                    verify=False,
                    region_name='eu-central-1')

                    
def video_to_frames(s3_objects, bucket):
    s3_client = get_s3_client()
    for s3_object in s3_objects:
        #exrect filename for temp path
        filename = s3_object.split("/",-1)[-1][:-4]
        filepath = s3_object
        tempfilename = s3_object.split("/",-1)[-1]
        temp_path = f'dataset/video/temp/{filename}/{tempfilename}'
        try:
            os.makedirs(f'dataset/video/temp/{filename}')
        except OSError:
            pass
        if exists(temp_path):
            print(f'File {temp_path} already exists. Skipping download.')
        else:
            #download video to temp path
            print(f'Downloading {filepath} to {temp_path}')
            s3_client.download_file(bucket, s3_object, temp_path)
        
        # Log the time
        time_start = time.time()
        # Start capturing the feed
        cap = cv2.VideoCapture(temp_path)
        # Find the number of frames
        video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
        print ("Number of frames: ", video_length)
        count = 0
        print ("Converting video..\n")
        # Start converting the video
        while cap.isOpened():
            # Extract the frame
            ret, frame = cap.read()
            frame = cv2.resize(frame, (1280, 720), Image.LANCZOS) # resize
            if not ret:
                continue
            # save image to temp folder
            cv2.imwrite(f'dataset/video/temp/{filename}/%#05d.jpg' % count, frame)
            # write image to S3 bucket
            Key = f'processed/images/{filename}%#05d.jpg' % count
            path_to_tempimage = f'dataset/video/temp/{filename}/%#05d.jpg' % count
            s3_client.upload_file(path_to_tempimage, bucket, Key)
            try:
                os.remove(path_to_tempimage)
            except OSError:
                pass
            # set frame count
            count += 30
            cap.set(cv2.CAP_PROP_POS_FRAMES, count) # skip x frames
            # If there are no more frames left
            if count > video_length:
                # Log the time again
                time_end = time.time()
                # Release the feed
                cap.release()
                # Print stats
                print (f"Done extracting frames from {filename}. {count} frames extracted")
                print (f"It took {time_end-time_start} seconds for conversion.")
                break

if __name__ == "__main__":
    bucket_name = 'my-bucket'
    s3_objects = ['raw/video/2022.07.17/myvideo.mp4']
    video_to_frames(s3_objects, bucket_name)

In [ ]:
### Labeln sie die Bilder in CVAT und exportieren sie Ihr Dataset ###

In [ ]:
### Unzip des heruntergeladenen Datensets und splitte es zufällig in Trainset, Validationset und Testset ###
import zipfile
import splitfolders

def extract_and_split(zip_file, output_folder, split_output_folder):
    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall(output_folder)
            print("Extracted all")
    except:
        print("Invalid file")
    try:
        os.mkdir(output_folder)
    except:
        print("Invalid file")
    try:
        os.mkdir(split_output_folder)
    except:
        print("Invalid file")
        
    splitfolders.ratio(output_folder, output=split_output_folder, seed=42, ratio=(.9, .1, .0), group_prefix=2)
    
    try :
        os.remove(output_folder)
    except OSError:
        pass

extract_and_split('Pfad/zu/zip', 'temp/Pfad', 'Pfad/zu/split/Dataset')

In [ ]:
### trainiere das vortrainierte Model yolov8x auf das Dataset, welches in customize/data/obj_cvat.yaml definiert ist ###

from ultralytics import YOLO
from PIL import Image
import cv2

model = YOLO("customize/model/yolov8x.yaml")
model.train(data="customize/data/obj_cvat.yaml", epochs=50, batch=2, device=0, seed=42, project='pfad/zu/ergebnis', name='test1', pretrained=True, imgsz=1280, save=True, workers=16)

In [ ]:
### nach dem training wird unter pfad/zu/ergebnis/weights eine best.pt datei erzeugt, dieses Model wird für eine Test Vorhersage genutzt ###

model = YOLO("pfad/zu/ergebnis/weights/best.pt")  # load a custom model
model.predict("pfad/zu/video.MP4", save=True, conf=0.5)

In [ ]:
### Durchsue S3 Speicher, lade die Videos herunter, treffe eine Vorhersage und speichere die ausgeschnittenen Bilder -> bereit für Segmentation ###

import boto3
import os
from ultralytics import YOLO
import configparser


def download_file_from_s3(s3_client, s3_bucket_name, s3_key, local_path):
    if os.path.exists(local_path):
        print(f'File {local_path} already exists, skipping download')
    else:
        print(f'Downloading {s3_key} from {s3_bucket_name}')
        # Download file
        s3_client.download_file(s3_bucket_name, s3_key, local_path)

def run_yolo_on_video(video_path):
    model = YOLO("runs/train/yolo8x6-1280-single-dolphin/12.02.23_pack/weights/best.pt")  # load a custom model
    model.predict(video_path, device=0, conf=0.8, save_txt=False, save_crop=True, vid_stride=10)

def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f'File {file_path} deleted')
    except OSError:
        print(f'Failed to delete file: {file_path}')

def filter_video_object_keys(s3_client, s3_bucket_name, prefix):
    objects = s3_client.list_objects_v2(Bucket=s3_bucket_name, Prefix=prefix)['Contents']
    video_objects = [obj['Key'] for obj in objects if obj['Key'].endswith(('.MOV', '.MP4'))]
    return video_objects

def main():
    s3_client = get_s3_client()
    s3_bucket_name = bucket_name
    s3_prefix = 'videos/oktober' # in welchen Ordnern soll nach Videodateien gesucht werden? kann auch eine einzelne datei sein
    local_video_dir = 'Temp_pfad'
    local_image_dir = 'Pfad/zu/ausgeschnittenen_Bildern'
    os.makedirs(local_video_dir, exist_ok=True)
    os.makedirs(local_image_dir, exist_ok=True)
    video_object_keys = filter_video_object_keys(s3_client, s3_bucket_name, s3_prefix)
    for object_key in video_object_keys:
        local_video_path = os.path.join(local_video_dir, os.path.basename(object_key))
        download_file_from_s3(s3_client, s3_bucket_name, object_key, local_video_path)
        run_yolo_on_video(local_video_path)
        delete_file(local_video_path)

if __name__ == '__main__':
    main()